# Initialization

In [4]:
#import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob,iglob
import pickle
import seaborn as sns
import os, sys
import subprocess as sp
#import MDAnalysis as mda
import panedr
from ase.io import read,write
from ase.visualize import view
from ase.io.trajectory import Trajectory
import nglview
from molmass import Formula
import matplotlib.gridspec as gridspec
from scipy.optimize import curve_fit
from scipy.interpolate import UnivariateSpline
#plt.style.use('hargu_seaborn.mplstyle')

def line(x,m,c):
    return m*x+c
##https://reader.elsevier.com/reader/sd/pii/S003238611630074X?token=964C73218EF46E4B1D21A7BFAA533DAF02A78AD3C68728A049B766A7725EDF3DB8D80FC98810D75D0381E9CAD9D415B7&originRegion=eu-west-1&originCreation=20210611120737
def xi(m1,c1,m2,c2):
    xi = (c1-c2) / (m2-m1)
    return xi
def yi(m1,c1,m2,c2):
    xi = (c1-c2) / (m2-m1)
    yi = m1 * xi + c1
    return 

SYSDIR='/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer'

## Initial ITP and topology files

In [3]:
%cd {SYSDIR}
#%mkdir MD analysis
#%mkdir MD/ITP MD/initial MD/scripts_mdp MD/PCL_PTM_OH MD/PCL_PTM_SLF
for egr in ['OH','SLF']:
    for nmon in [16,80]:
        #%mkdir MD/PCL_PTM_{egr}/{nmon}mer
        for conc in [0,0.125]:
            #%mkdir {SYSDIR}/MD/PCL_PTM_{egr}/{nmon}mer/{conc}
            %cd {SYSDIR}/MD/PCL_PTM_{egr}/{nmon}mer/{conc}
            %mkdir 500 #em preeq Tg 400 450
    

/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0.125
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/80mer/0
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/80mer/0.125
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/16mer/0
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/16mer/0.125
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/80mer/0
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/80mer/0.125


The single polymer chains are built with GroPolBul (scripts using ambertools and acpype) are in folder /cfs/klemming/nobackup/h/hargu978/GroPolBul/polymers/*

Below we can visualize the polymers

In [68]:
for pgro in ['PCL80_PTM20_SLF']: #,'PCL80_PTM20_OH','PCL16_PTM4_SLF','PCL80_PTM20_SLF'
    %cd /cfs/klemming/nobackup/h/hargu978/GroPolBul/polymers/{pgro}/md
    pgro=read('npt.gro')
    vi=view(pgro,viewer='ngl')
    print(pgro.get_chemical_formula());print(Formula(pgro.get_chemical_formula()).mass)
    maxa=pgro.get_number_of_atoms()
    print(pgro.get_distance(1,maxa-5))
vi

/cfs/klemming/nobackup/h/hargu978/GroPolBul/polymers/PCL80_PTM20_SLF/md
C588H964N10O228
11821.901614
63.72721475162711


/afs/pdc.kth.se/home/h/hargu978/.local/lib/python3.6/site-packages/ase/atoms.py:968: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)


In [54]:
#####COPYING ITP files and initial pdb files #######
#Run this commond by chnagig the folders and file names to copy the ITP files
POLDIR='/cfs/klemming/nobackup/h/hargu978/GroPolBul/polymers'
MDDIR='/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD'
for nmon in [16,80]:
    egr='OH'
    pol='PCL'+str(nmon)+'_PTM'+str(int(nmon/4))+'_'+str(egr)
    !sed -n -e '/^\s*\[\s*system\s*\]\s*$/q' -e '/^\s*\[\s*moleculetype\s*\]\s*$/,$p' {POLDIR}/{pol}/polymer_GMX.top > {MDDIR}/ITP/{pol}.itp
    !cp {POLDIR}/{pol}/initial.pdb {MDDIR}/PCL_PTM_{egr}/{nmon}mer/single_pol.pdb
    ##if needed substitue the HPT and TPT res names to PCL
    !sed -i -e 's/HPT/PTM/g' -e 's/TPT/PTM/g' -e 's/CLL/PCL/g' -e 's/CLR/PCL/g' {MDDIR}/ITP/{pol}.itp {MDDIR}/PCL_PTM_{egr}/{nmon}mer/single_pol.pdb
    print(pol)
    !head {MDDIR}/ITP/{pol}.itp
    egr='SLF'
    pol='PCL'+str(nmon)+'_PTM'+str(int(nmon/4))+'_'+str(egr)
    !sed -n -e '/^\s*\[\s*system\s*\]\s*$/q' -e '/^\s*\[\s*moleculetype\s*\]\s*$/,$p' {POLDIR}/{pol}/polymer_GMX.top > {MDDIR}/ITP/{pol}.itp
    !cp {POLDIR}/{pol}/initial.pdb {MDDIR}/PCL_PTM_{egr}/{nmon}mer/single_pol.pdb
    ##if needed substitue the HPT and TPT res names to PCL
    !sed -i -e 's/HPT/PTM/g' -e 's/TPT/PTM/g' -e 's/CLL/PCL/g' -e 's/CLR/PCL/g' {MDDIR}/ITP/{pol}.itp {MDDIR}/PCL_PTM_{egr}/{nmon}mer/single_pol.pdb
    ##After copying check the files and make sure that all the res names are PCL/PTM/SLF or desired abbrevations (to remove confusion in analysis section)
    print(pol)
    !head {MDDIR}/ITP/{pol}.itp
    

PCL16_PTM4_OH
[ moleculetype ]
;name            nrexcl
 polymer          3

[ atoms ]
;   nr  type  resi  res  atom  cgnr     charge      mass       ; qtot   bond_type
     1   oh     1   PTM     O    1    -0.590775     16.00000 ; qtot -0.591
     2   ho     1   PTM   H10    2     0.403882      1.00800 ; qtot -0.187
     3   c3     1   PTM     C    3     0.133185     12.01000 ; qtot -0.054
     4   h1     1   PTM     H    4     0.036666      1.00800 ; qtot -0.017
PCL16_PTM4_SLF
[ moleculetype ]
;name            nrexcl
 polymer          3

[ atoms ]
;   nr  type  resi  res  atom  cgnr     charge      mass       ; qtot   bond_type
     1    c     1   PTM     C    1     0.737469     12.01000 ; qtot 0.737
     2    n     1   PTM     N    2    -0.541152     14.01000 ; qtot 0.196
     3   c3     1   PTM    C1    3     0.096309     12.01000 ; qtot 0.293
     4   c3     1   PTM    C2    4    -0.092102     12.01000 ; qtot 0.201
PCL80_PTM20_OH
[ moleculetype ]
;name            nrexcl
 polymer   

In [20]:
### Make the topology file with ITP directory location and system details
##Details
## pol nmon/pol npol total.nmon
# 16:4   20      30    600
# 80:20  100     6     600

MDDIR='/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD'

for egr in ['OH','SLF']:
    for nmon,npol in zip([16,80],[30,6]):        
        for conc in [0,0.125]:
            %cd {MDDIR}/PCL_PTM_{egr}/{nmon}mer/{conc}
            pol='PCL'+str(nmon)+'_PTM'+str(int(nmon/4))+'_'+str(egr)
            topol=open('topol.top','w+')
            topol.write('''#include "'''+str(MDDIR)+'''/ITP/ff.itp"
#include "'''+str(MDDIR)+'''/ITP/'''+str(pol)+'''.itp"    
#include "'''+str(MDDIR)+'''/ITP/li_75c.itp"  
#include "'''+str(MDDIR)+'''/ITP/tfsi_75c.itp"
        
[ system ] 
'''+str(pol)+'''_'''+str(conc)+'''

[ molecules ]
polymer   '''+str(npol)+'''     
LI    '''+str(int(npol*nmon*(5/4)*float(conc)))+'''
TFS   '''+str(int(npol*nmon*(5/4)*float(conc)))+'''
''')
            topol.close() 
            
#        conc=0;
#        %cd {MDDIR}/PCL_PTM_{egr}/{nmon}mer/{conc}
#        pol='PCL'+str(nmon)+'_PTM'+str(int(nmon/4))+'_'+str(egr)
#        topol=open('topol.top','w+')
#        topol.write('''
##include '''+str(MDDIR)+'''/ITP/ff.itp
##include '''+str(MDDIR)+'''/ITP/'''+str(pol)+'''.itp    
##include '''+str(MDDIR)+'''/ITP/li_75c.itp  
##include '''+str(MDDIR)+'''/ITP/tfsi_75c.itp
#       
#[ system ] 
#'''+str(pol)+'''_'''+str(conc)+'''
#
#[ molecules ]
#PCL   '''+str(npol*nmon)+'''
#PTM   '''+str(int(npol*nmon/4))+'''     
#''')
#        topol.close() 


/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0.125
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/80mer/0
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/80mer/0.125
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/16mer/0
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/16mer/0.125
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/80mer/0
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/80mer/0.125


## Simulation boxes (packmol)

In [10]:
#Build initial simulation boxes using packmol
##Details
## pol nmon/pol npol total.nmon
# 16:4   20      30    600
# 80:20  100     6     600

MDDIR='/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD'

for egr in ['SLF']: #'OH',
    for nmon,npol in zip([16,80],[30,6]):
        pol='PCL'+str(nmon)+'_PTM'+str(int(nmon/4))+'_'+str(egr)
        conc=0;
        %cd {MDDIR}/PCL_PTM_{egr}/{nmon}mer/{conc}
        pack=open('packmol.inp','w+')
        pack.write('''tolerance 2.0
filetype pdb
output initial.pdb
structure ../single_pol.pdb
  number '''+str(npol)+'''
  inside cube 0. 0. 0. 100. 
end structure
        ''')
        pack.close()
        #!{MDDIR}/packmol < packmol.inp
        
        conc=0.125;
        %cd {MDDIR}/PCL_PTM_{egr}/{nmon}mer/{conc}
        pack=open('packmol.inp','w+')
        pack.write('''tolerance 2.0
filetype pdb
output initial.pdb
structure ../single_pol.pdb
  number '''+str(npol)+'''
  inside cube 0. 0. 0. 100. 
end structure

structure '''+str(MDDIR)+'''/li.pdb
  number '''+str(int(npol*nmon*(5/4)*float(conc)))+'''
  inside cube 20. 20. 20. 50.
end structure

structure '''+str(MDDIR)+'''/tfsi.pdb
  number '''+str(int(npol*nmon*(5/4)*float(conc)))+'''
  inside cube 20. 20. 20. 50.
end structure
        ''')
        pack.close()
        !{MDDIR}/packmol < packmol.inp


/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/16mer/0
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/16mer/0.125

################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 17.333 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: www.ime.unicamp.br/~martinez/packmol 

  Reading input file... (Control-C aborts)
  Seed for random number generator:      1234567
  Output file: initial.pdb
  Reading coordinate file: ../single_pol.pdb
  Reading coordinate file: /cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/li.pdb
  Reading coordinate file: /cfs/klemming/nobackup/h/hargu978/Cuc_kri

          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .32938E+04
  Best function value before: f = .48718E+04
  Improvement from best function value:    32.39 %
  Improvement from last loop:    32.39 %
  Maximum violation of target distance:     3.989807
  Maximum violation of the constraints: .93596E+01
  All-type function value: .10089E+05

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

  Starting GENCAN loop:            2
  Scaling radii by:    1.1000000000000001     

  Packing:|0                                                             100%|
          |******************************************************************|
          |******************************************************************|

  Function value f

          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .16531E+03
  Best function value before: f = .55448E+03
  Improvement from best function value:    70.19 %
  Improvement from last loop:    66.74 %
  Maximum violation of target distance:     3.502082
  Maximum violation of the constraints: .34443E+00
  All-type function value: .73029E+04

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

  Starting GENCAN loop:           12
  Scaling radii by:    1.0000000000000000     

  Packing:|0                                                             100%|
          |******************************************************************|
          |******************************************************************|

  Function value f

          |******************************************************************|

  Function value from last GENCAN loop: f = .14604E+03
  Best function value before: f = .45157E+02
  Improvement from best function value:   -99.99 %
  Improvement from last loop:     9.72 %
  Maximum violation of target distance:     3.817301
  Maximum violation of the constraints: .80847E+00
  All-type function value: .66187E+04

--------------------------------------------------------------------------------

  Current solution written to file: initial.pdb
  Moving worst molecules ... 
  Function value before moving molecules:   146.04110241612486     
  Type         1 molecules with non-zero contributions:   70.00%
  Moving         1 molecules of type         1
  New positions will be based on good molecules (movebadrandom is not set) 
   Moving:|0                                                             100%|
          |******************************************************************|
  Function 

          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .83391E+03
  Best function value before: f = .18312E+04
  Improvement from best function value:    54.46 %
  Improvement from last loop:    54.46 %
  Maximum violation of target distance:     3.788509
  Maximum violation of the constraints: .10685E+01

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

  Starting GENCAN loop:            2
  Scaling radii by:    1.1000000000000001     

  Packing:|0                                                             100%|
          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .49593E+03
 

...


--------------------------------------------------------------------------------

  Starting GENCAN loop:           12
  Scaling radii by:    1.0000000000000000     

  Packing:|0                                                             100%|
          |******************************************************************|
          |****************************

  Function value from last GENCAN loop: f = .27534E-02
  Best function value before: f = .11039E-01
  Improvement from best function value:    75.06 %
  Improvement from last loop:    75.06 %
  Maximum violation of target distance:     0.008976
  Maximum violation of the constraints: .23172E-02
  Current structure written to file: initial.pdb
--------------------------------------------------------------------------------
  Packing solved for molecules of type           1
  Objective function value:    2.7533929587360770E-003
  Maximum violation of target distance:    8.9759985000936737E-003
  Max. constraint violation:    2.

          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .17968E+04
  Best function value before: f = .39539E+03
  Improvement from best function value:   -99.99 %
  Improvement from last loop:    39.55 %
  Maximum violation of target distance:     3.963417
  Maximum violation of the constraints: .22716E+00

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

  Starting GENCAN loop:            6
  Scaling radii by:    1.0000000000000000     

  Packing:|0                                                             100%|
          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .17161E+04
 

          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .10939E+04
  Best function value before: f = .39539E+03
  Improvement from best function value:   -99.99 %
  Improvement from last loop:    25.61 %
  Maximum violation of target distance:     3.926950
  Maximum violation of the constraints: .52314E+00

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

  Starting GENCAN loop:           14
  Scaling radii by:    1.0000000000000000     

  Packing:|0                                                             100%|
          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .60772E+03
 

          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .38108E+03
  Best function value before: f = .13698E+03
  Improvement from best function value:   -99.99 %
  Improvement from last loop:    66.68 %
  Maximum violation of target distance:     3.973839
  Maximum violation of the constraints: .16223E+01

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

  Starting GENCAN loop:           22
  Scaling radii by:    1.0000000000000000     

  Packing:|0                                                             100%|
          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .83697E+02
 

          |******************************************************************|

  Function value from last GENCAN loop: f = .37488E+02
  Best function value before: f = .18655E+01
  Improvement from best function value:   -99.99 %
  Improvement from last loop:    99.39 %
  Maximum violation of target distance:     3.198842
  Maximum violation of the constraints: .14611E+01

--------------------------------------------------------------------------------

  Current solution written to file: initial.pdb

--------------------------------------------------------------------------------

  Starting GENCAN loop:           30
  Scaling radii by:    1.0000000000000000     

  Packing:|0                                                             100%|
          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .52332E+00
  Best function value before: f 

In [11]:
MDDIR='/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD'

%cd {MDDIR}/PCL_PTM_SLF/80mer/0.125
vi=view(read('initial.pdb'),viewer='ngl')
vi

/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/80mer/0.125


# MD simulations (Gromacs)

## Energy minimization

In [21]:
%%bash
#RUN in beskow/tegner terminal for preparing initial configuration file and EM step

module load gromacs/2021-avx
MDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD

for egr in OH SLF;do
    for nmon in 16 80;do
        for conc in 0 0.125;do
            cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/
            gmx editconf -f initial.pdb -o preem.gro -c -bt cubic -d 0
            mkdir em
            cd em
            echo '''; em.mdp - used as input into grompp to generate em.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
dt          = 0.001	    ; ps
emtol       = 100.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 100000          ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
rlist           = 1.3
cutoff-scheme   = Verlet    ; Buffered neighbor searching
;coulombtype    = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.3       ; Short-range electrostatic cut-off
vdwtype         = cut-off
rvdw            = 1.3       ; Short-range Van der Waals cut-off
fourierspacing  = 0.16
;pme_order      = 4
ewald_rtol      = 1e-4
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions
''' > em.mdp

        gmx grompp -f em.mdp -c ../preem.gro -p ../topol.top -o em.tpr -maxwarn 2
        gmx mdrun -s em.tpr -deffnm em -v
        #tail em.log
        rm \#*
        cd ..
        rm \#*
        done
    done
done


Process is interrupted.


In [3]:
%%bash
#RUN in beskow/tegner terminal for preparing initial configuration file and EM step

module load gromacs/2018.3
MDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD

for egr in OH SLF;do
    for nmon in 16 80;do
        for conc in 0 0.125;do
            cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/
            gmx_seq grompp -f em/em.mdp -c preem.gro -p topol.top -o analysis.tpr -maxwarn 2
        done
    done
done


Analysing residue names:
There are:   300      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 1 Mb of data
Analysing residue names:
There are:   375      Other residues
There are:    75        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 1 Mb of data
Analysing residue names:
There are:   300      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 1 Mb of data
Analysing residue names:
There are:   375      Other residues
There are:    75        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will gene

                      :-) GROMACS - gmx grompp, 2018.3 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS de

In [27]:
for egr in ['OH','SLF']: #
    for nmon,npol in zip([16,80],[30,6]):
        for conc in [0, 0.125]:
            %cd {MDDIR}/PCL_PTM_{egr}/{nmon}mer/{conc}/em
            !tail -20 em.log


/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0/em
           1375     1375.00000

   Energies (kJ/mol)
           Bond          Angle    Proper Dih. Ryckaert-Bell.          LJ-14
    4.01644e+02    2.48288e+03    3.93107e+01    1.24010e+04    6.67118e+03
     Coulomb-14        LJ (SR)   Coulomb (SR)      Potential Pressure (bar)
   -4.44572e+04   -1.09554e+04   -9.78902e+03   -4.32056e+04   -6.12580e+01


Steepest Descents converged to Fmax < 100 in 1376 steps
Potential Energy  = -4.3205605e+04
Maximum force     =  8.4005898e+01 on atom 4567
Norm of force     =  7.5333917e+00

    D O M A I N   D E C O M P O S I T I O N   S T A T I S T I C S

 av. #atoms communicated per step for force:  2 x 7197.9

Finished mdrun on rank 0 Thu Dec 30 17:00:44 2021

/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0.125/em
          21709    21709.00000

   Energies (kJ/mol)
           Bond          Angle    Proper Dih. Ryc

 ## Pre-equilibration step

In [92]:
%%bash
#RUN in beskow terminal for preeq step
#module swap gcc/8.2.0
module load gromacs/2021-avx
MDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD

for egr in SLF;do # SLF OH
    for nmon in 80;do #16 80
        for conc in 0;do #0 0.125
            cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/preeq
            echo PCL_PTM_$egr/${nmon}mer/$conc
            sed -e 's/egrp/'${egr}'/g' -e 's/nmonv/'${nmon}'/g' -e 's/conc/'${conc}'/g' $MDDIR/scripts_mdp/preeq.sh > preeq.sh
            top=$MDDIR/PCL_PTM_${egr}/${nmon}mer/${conc}/topol.top
            mdp=$MDDIR/scripts_mdp/nvt_preeq.mdp
            #gmx grompp -f $mdp -c ../em/em.gro -p $top -o nvt.tpr -maxwarn 2

            mdp=$MDDIR/scripts_mdp/npt_preeq.mdp
            #gmx editconf -f nvt.gro -o nvt.gro -d 0.1 -bt cubic
            #gmx grompp -f $mdp -c nvt.gro -p $top -o nvt.tpr -maxwarn 2

            #sed -i 's/nodes=4/nodes=4/' preeq.sh
            sed -i 's/node=32/node=12/' preeq.sh
            #sed -i 's/24:00:00/10:00:00/' preeq.sh
            sub=$(sbatch preeq.sh);id=$(grep -Eo '[0-9]+' <<<"$sub");echo "job submitted at ${tk[t]}: $id"
            #for s in {1..3};do
                #depend=$(sbatch --dependency=afterany:$id preeq.sh);id=$(grep -Eo '[0-9]+' <<<"$depend");
                #echo "job submitted with dependency $s"
            #done
            
            rm \#*
        done
    done
done


PCL_PTM_SLF/80mer/0
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 10740 atoms
Volume: 1348.43 nm^3, corresponds to roughly 606700 electrons
Velocities found
    system size :  6.228  6.559  6.402 (nm)
    diameter    :  7.652               (nm)
    center      :  5.541  5.504  5.525 (nm)
    box vectors : 11.048 11.048 11.048 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :1348.43               (nm^3)
    shift       : -1.615 -1.578 -1.599 (nm)
new center      :  3.926  3.926  3.926 (nm)
new box vectors :  7.852  7.852  7.852 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  : 484.08               (nm^3)
Setting the LD random seed to -1170219077

Generated 210 of the 210 non-bonded parameter combinations

Generated 210 of the 210 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'polymer'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 

gcc/8.2.0(22):ERROR:150: Module 'gcc/8.2.0' conflicts with the currently loaded module(s) 'gcc/7.2.0'
gcc/8.2.0(22):ERROR:102: Tcl command execution failed: conflict gcc

openmpi/4.0-gcc-8.2(12):ERROR:151: Module 'openmpi/4.0-gcc-8.2' depends on one of the module(s) 'gcc/8.2.0'
openmpi/4.0-gcc-8.2(12):ERROR:102: Tcl command execution failed: prereq gcc/8.2.0

                      :-) GROMACS - gmx editconf, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Chr

In [10]:
MDDIR='/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD'
for egr in ['OH','SLF']: #
    for nmon,npol in zip([16,80],[30,6]):
        for conc in [0, 0.125]:
            %cd {MDDIR}/PCL_PTM_{egr}/{nmon}mer/{conc}/preeq
            !tail -3 preeq_nvt_npt.out
            !tail -15 npt.log
            #!ls *.gro


/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0/preeq
GROMACS reminds you: "Baby, It Aint Over Till It's Over" (Lenny Kravitz)

rm: cannot remove '#*': No such file or directory
 Listed buffer ops.     4    1    5284101      58.810        541.049   0.2
 Nonbonded F kernel     4    1   10568202   14053.042     129287.046  58.6
 Nonbonded F clear      4    1   10568202      16.571        152.449   0.1
 NB X buffer ops.       4    1   10462518      86.371        794.607   0.4
 NB F buffer ops.       4    1    5284101      56.455        519.387   0.2
 Clear force buffer     4    1    5284101      15.971        146.931   0.1
-----------------------------------------------------------------------------

               Core t (s)   Wall t (s)        (%)
       Time:    95969.935    23992.508      400.0
                         6h39:52
                 (ns/day)    (hour/ns)
Performance:       19.029        1.261
Finished mdrun on rank 0 Fri Dec 31 21:26

In [3]:
MDDIR='/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD'

%cd {MDDIR}/PCL_PTM_SLF/80mer/0/preeq
vi=view(read('npt.gro'),viewer='ngl')
vi

/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_SLF/80mer/0/preeq


## Equilibiriation and Production step

In [5]:
%%bash
#RUN in beskow terminal for equilibiration step
#module swap gcc/8.2.0
module load gromacs/2021-avx
MDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD

for egr in OH SLF;do #OH SLF
    for nmon in 16 80;do #80
        for conc in 0 0.125;do #0 0.125
            for temp in 400 500;do
            
                cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/$temp
                echo PCL_PTM_$egr/${nmon}mer/$conc/$temp
                sed -e 's/egrp/'${egr}'/g' -e 's/nmonv/'${nmon}'/g' -e 's/conc/'${conc}'/g' -e 's/temp./'${temp}'/g' $MDDIR/scripts_mdp/eq_prod_T.sh > eq_prod_T.sh
                sed -e 's/temp./'${temp}'/g' $MDDIR/scripts_mdp/npt_eq_T.mdp > npt_eq_T.mdp
                sed -e 's/temp./'${temp}'/g' $MDDIR/scripts_mdp/npt_prod_T.mdp > npt_prod_T.mdp
                top=$MDDIR/PCL_PTM_${egr}/${nmon}mer/${conc}/topol.top
                #gmx grompp -f npt_eq_T.mdp -c ../preeq/npt.gro -p $top -o npt_eq.tpr -maxwarn 2
                #sed -i 's/nodes=8/nodes=4/' eq_prod_T.sh
                #sed -i 's/node=32/node=12/' eq_prod_T.sh
                sub=$(sbatch eq_prod_T.sh);id=$(grep -Eo '[0-9]+' <<<"$sub");echo "job submitted at ${tk[t]}: $id"
                #for s in {1..3};do
                    #depend=$(sbatch --dependency=afterany:$id preeq.sh);id=$(grep -Eo '[0-9]+' <<<"$depend");
                    #echo "job submitted with dependency $s"
                #done
                rm \#*
            done
        done
    done
done


PCL_PTM_OH/16mer/0/500
Setting the LD random seed to -1124237441

Generated 210 of the 210 non-bonded parameter combinations

Generated 210 of the 210 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'polymer'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'LI'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'TFS'

turning H bonds into constraints...
Analysing residue names:
There are:   300      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 500 K

Calculated rlist for 1x1 atom pair-list as 1.200 nm, buffer size 0.000 nm

Set rlist, assuming 4x4 atom pair-list, to 1.200 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 12x12x12, spacing 0.398 0.398 0.398

E

gcc/8.2.0(22):ERROR:150: Module 'gcc/8.2.0' conflicts with the currently loaded module(s) 'gcc/7.2.0'
gcc/8.2.0(22):ERROR:102: Tcl command execution failed: conflict gcc

openmpi/4.0-gcc-8.2(12):ERROR:151: Module 'openmpi/4.0-gcc-8.2' depends on one of the module(s) 'gcc/8.2.0'
openmpi/4.0-gcc-8.2(12):ERROR:102: Tcl command execution failed: prereq gcc/8.2.0

                       :-) GROMACS - gmx grompp, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Chri

CalledProcessError: Command 'b'#RUN in beskow terminal for equilibiration step\n#module swap gcc/8.2.0\nmodule load gromacs/2021-avx\nMDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD\n\nfor egr in OH SLF;do #OH SLF\n    for nmon in 16 80;do #80\n        for conc in 0 0.125;do #0 0.125\n            for temp in 500;do\n            \n                cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/$temp\n                echo PCL_PTM_$egr/${nmon}mer/$conc/$temp\n                sed -e \'s/egrp/\'${egr}\'/g\' -e \'s/nmonv/\'${nmon}\'/g\' -e \'s/conc/\'${conc}\'/g\' -e \'s/temp./\'${temp}\'/g\' $MDDIR/scripts_mdp/eq_prod_T.sh > eq_prod_T.sh\n                sed -e \'s/temp./\'${temp}\'/g\' $MDDIR/scripts_mdp/npt_eq_T.mdp > npt_eq_T.mdp\n                sed -e \'s/temp./\'${temp}\'/g\' $MDDIR/scripts_mdp/npt_prod_T.mdp > npt_prod_T.mdp\n                top=$MDDIR/PCL_PTM_${egr}/${nmon}mer/${conc}/topol.top\n                gmx grompp -f npt_eq_T.mdp -c ../preeq/npt.gro -p $t

In [4]:
%%bash
#RUN in beskow terminal for production step
#module swap gcc/8.2.0

MDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD

for egr in SLF;do
    for nmon in 16;do #80
        for conc in 0 0.125;do
            for temp in 400 ;do
            
                cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/$temp
                echo PCL_PTM_$egr/${nmon}mer/$conc/$temp
                sed -e 's/egrp/'${egr}'/g' -e 's/nmonv/'${nmon}'/g' -e 's/conc/'${conc}'/g' -e 's/temp./'${temp}'/g' $MDDIR/scripts_mdp/prod_T.sh > prod_T.sh
                sed -e 's/temp./'${temp}'/g' $MDDIR/scripts_mdp/npt_prod_T.mdp > npt_prod_T.mdp
                top=$MDDIR/PCL_PTM_${egr}/${nmon}mer/${conc}/topol.top
                #gmx grompp -f npt_prod_T.mdp -c npt_eq.gro -p $top -o npt_prod.tpr -maxwarn 2

                #sed -i 's/nodes=4/nodes=4/' prod_T.sh
                #sed -i 's/node=32/node=12/' prod_T.sh
                sub=$(sbatch prod_T.sh);id=$(grep -Eo '[0-9]+' <<<"$sub");echo "job submitted at ${tk[t]}: $id"
                #for s in {1..3};do
                    #depend=$(sbatch --dependency=afterany:$id prod_T.sh);id=$(grep -Eo '[0-9]+' <<<"$depend");
                    #echo "job submitted with dependency $s"
                #done
                rm \#*
            done
        done
    done
done


PCL_PTM_SLF/16mer/0/400
job submitted at : 139925
PCL_PTM_SLF/16mer/0.125/400
job submitted at : 139926


rm: cannot remove '#*': No such file or directory
rm: cannot remove '#*': No such file or directory


CalledProcessError: Command 'b'#RUN in beskow terminal for production step\n#module swap gcc/8.2.0\n\nMDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD\n\nfor egr in SLF;do\n    for nmon in 16;do #80\n        for conc in 0 0.125;do\n            for temp in 400 ;do\n            \n                cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/$temp\n                echo PCL_PTM_$egr/${nmon}mer/$conc/$temp\n                sed -e \'s/egrp/\'${egr}\'/g\' -e \'s/nmonv/\'${nmon}\'/g\' -e \'s/conc/\'${conc}\'/g\' -e \'s/temp./\'${temp}\'/g\' $MDDIR/scripts_mdp/prod_T.sh > prod_T.sh\n                sed -e \'s/temp./\'${temp}\'/g\' $MDDIR/scripts_mdp/npt_prod_T.mdp > npt_prod_T.mdp\n                top=$MDDIR/PCL_PTM_${egr}/${nmon}mer/${conc}/topol.top\n                #gmx grompp -f npt_prod_T.mdp -c npt_eq.gro -p $top -o npt_prod.tpr -maxwarn 2\n\n                #sed -i \'s/nodes=4/nodes=4/\' prod_T.sh\n                #sed -i \'s/node=32/node=12/\' prod_T.sh\n           

In [2]:
MDDIR='/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD'
for egr in ['OH','SLF']: #
    for nmon,npol in zip([16,80],[30,6]):
        for conc in [0, 0.125]:
            %cd {MDDIR}/PCL_PTM_{egr}/{nmon}mer/{conc}/400
            !tail -3 npt_eq_prod.out
            !tail -13 npt_prod.log 
            !ls *.gro


/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0/400
imb F 12% step 340929800, will finish Sun Feb 27 05:22:15 2022
imb F 12% step 340929900, will finish Sun Feb 27 05:22:14 2022
imb F 13% step 340930000, will finish Sun Feb 27 05:22:14 2022
           Step           Time
      340930000   340930.00000

   Energies (kJ/mol)
           Bond          Angle    Proper Dih. Ryckaert-Bell.          LJ-14
    8.15931e+03    2.67481e+04    8.55984e+02    1.70815e+04    8.44338e+03
     Coulomb-14        LJ (SR)  Disper. corr.   Coulomb (SR)   Coul. recip.
   -4.51998e+04   -2.54177e+04   -1.97388e+03   -1.22565e+04    4.34043e+02
      Potential    Kinetic En.   Total Energy  Conserved En.    Temperature
   -2.31256e+04    4.35533e+04    2.04277e+04    3.62340e+06    4.02835e+02
 Pres. DC (bar) Pressure (bar)   Constr. rmsd
   -2.99161e+02    5.67557e+02    9.00902e-07

npt_eq.gro  npt_prod.gro
/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_p

## Glass Transition temperature

In [8]:
for i in reversed(range(2,23)):
    print(i*20+100,i*20+100,end =" ")
print('\n')
for i in range(2,23):
    print(i*20+100,i*20+100,end =" ")

print('\n')
for i in range(2,23):
    print((i-2)*4000+1000,(i-2+1)*4000,end =" ")

540 540 520 520 500 500 480 480 460 460 440 440 420 420 400 400 380 380 360 360 340 340 320 320 300 300 280 280 260 260 240 240 220 220 200 200 180 180 160 160 140 140 

140 140 160 160 180 180 200 200 220 220 240 240 260 260 280 280 300 300 320 320 340 340 360 360 380 380 400 400 420 420 440 440 460 460 480 480 500 500 520 520 540 540 

1000 4000 5000 8000 9000 12000 13000 16000 17000 20000 21000 24000 25000 28000 29000 32000 33000 36000 37000 40000 41000 44000 45000 48000 49000 52000 53000 56000 57000 60000 61000 64000 65000 68000 69000 72000 73000 76000 77000 80000 81000 84000 

In [13]:
%%bash
#RUN in beskow terminal for calculating Tg 
#module swap gcc/8.2.0
module load gromacs/2021-avx
MDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD

for egr in OH SLF;do #OH 
    for nmon in 16 80 ;do #80
        for conc in 0 0.125;do #0 0.125
            
            cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/Tg
            echo PCL_PTM_$egr/${nmon}mer/$conc
            sed -e 's/egrp/'${egr}'/g' -e 's/nmonv/'${nmon}'/g' -e 's/conc/'${conc}'/g' $MDDIR/scripts_mdp/Tg_540_140K.sh > Tg_540_140K.sh
            mdp=$MDDIR/scripts_mdp/npt_Tg.mdp
            top=$MDDIR/PCL_PTM_${egr}/${nmon}mer/${conc}/topol.top
            #gmx grompp -f $mdp -c ../400/npt_eq.gro -p $top -o npt.tpr -maxwarn 2
            
            #sed -i 's/nodes=4/nodes=4/' Tg_540_140K.sh
            #sed -i 's/node=32/node=12/' Tg_540_140K.sh
            sub=$(sbatch Tg_540_140K.sh);id=$(grep -Eo '[0-9]+' <<<"$sub");echo "job submitted at ${tk[t]}: $id"
            rm \#*
            
        done
    done
done


PCL_PTM_OH/16mer/0
Setting the LD random seed to 2145287133

Generated 210 of the 210 non-bonded parameter combinations

Generated 210 of the 210 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'polymer'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'LI'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'TFS'

turning H bonds into constraints...
Analysing residue names:
There are:   300      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 540 K

Calculated rlist for 1x1 atom pair-list as 1.200 nm, buffer size 0.000 nm

Set rlist, assuming 4x4 atom pair-list, to 1.200 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 12x12x12, spacing 0.398 0.398 0.398

Estima

gcc/8.2.0(22):ERROR:150: Module 'gcc/8.2.0' conflicts with the currently loaded module(s) 'gcc/7.2.0'
gcc/8.2.0(22):ERROR:102: Tcl command execution failed: conflict gcc

openmpi/4.0-gcc-8.2(12):ERROR:151: Module 'openmpi/4.0-gcc-8.2' depends on one of the module(s) 'gcc/8.2.0'
openmpi/4.0-gcc-8.2(12):ERROR:102: Tcl command execution failed: prereq gcc/8.2.0

                       :-) GROMACS - gmx grompp, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Chri

In [9]:
%%bash
#RUN in beskow terminal for calculating Tg # Heating and then cooling
#module swap gcc/8.2.0
module load gromacs/2021-avx
MDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD

for egr in OH SLF;do #OH 
    for nmon in 16 80 ;do #80
        for conc in 0 0.125;do #0 0.125
            
            cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/Tg
            echo PCL_PTM_$egr/${nmon}mer/$conc
            sed -e 's/egrp/'${egr}'/g' -e 's/nmonv/'${nmon}'/g' -e 's/conc/'${conc}'/g' -e 's/npt/npt_heating/g' $MDDIR/scripts_mdp/Tg_540_140K.sh > Tg_540_140K.sh
            mdp=$MDDIR/scripts_mdp/npt_Tg_heating.mdp
            top=$MDDIR/PCL_PTM_${egr}/${nmon}mer/${conc}/topol.top
            #gmx grompp -f $mdp -c npt.gro -p $top -o npt_heating.tpr -maxwarn 2
            mdp=$MDDIR/scripts_mdp/npt_Tg_cooling.mdp
            #gmx grompp -f $mdp -c npt_heating.gro -p $top -o npt_cooling.tpr -maxwarn 2
            #sed -i 's/nodes=4/nodes=4/' Tg_540_140K.sh
            #sed -i 's/node=32/node=12/' Tg_540_140K.sh
            sub=$(sbatch Tg_540_140K.sh);id=$(grep -Eo '[0-9]+' <<<"$sub");echo "job submitted at ${tk[t]}: $id"
            rm \#*
            
        done
    done
done


PCL_PTM_OH/16mer/0
Setting the LD random seed to -1085957

Generated 210 of the 210 non-bonded parameter combinations

Generated 210 of the 210 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'polymer'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'LI'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'TFS'

turning H bonds into constraints...
Analysing residue names:
There are:   300      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 540 K

Calculated rlist for 1x1 atom pair-list as 1.200 nm, buffer size 0.000 nm

Set rlist, assuming 4x4 atom pair-list, to 1.200 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 12x12x12, spacing 0.385 0.385 0.385

Estimate

gcc/8.2.0(22):ERROR:150: Module 'gcc/8.2.0' conflicts with the currently loaded module(s) 'gcc/7.2.0'
gcc/8.2.0(22):ERROR:102: Tcl command execution failed: conflict gcc

openmpi/4.0-gcc-8.2(12):ERROR:151: Module 'openmpi/4.0-gcc-8.2' depends on one of the module(s) 'gcc/8.2.0'
openmpi/4.0-gcc-8.2(12):ERROR:102: Tcl command execution failed: prereq gcc/8.2.0

                       :-) GROMACS - gmx grompp, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Chri

CalledProcessError: Command 'b'#RUN in beskow terminal for calculating Tg # Heating and then cooling\n#module swap gcc/8.2.0\nmodule load gromacs/2021-avx\nMDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD\n\nfor egr in OH SLF;do #OH \n    for nmon in 16 80 ;do #80\n        for conc in 0 0.125;do #0 0.125\n            \n            cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/Tg\n            echo PCL_PTM_$egr/${nmon}mer/$conc\n            sed -e \'s/egrp/\'${egr}\'/g\' -e \'s/nmonv/\'${nmon}\'/g\' -e \'s/conc/\'${conc}\'/g\' -e \'s/npt/npt_heating/g\' $MDDIR/scripts_mdp/Tg_540_140K.sh > Tg_540_140K.sh\n            mdp=$MDDIR/scripts_mdp/npt_Tg_heating.mdp\n            top=$MDDIR/PCL_PTM_${egr}/${nmon}mer/${conc}/topol.top\n            gmx grompp -f $mdp -c npt.gro -p $top -o npt_heating.tpr -maxwarn 2\n            \n            #sed -i \'s/nodes=4/nodes=4/\' Tg_540_140K.sh\n            #sed -i \'s/node=32/node=12/\' Tg_540_140K.sh\n            #sub=$(sbatch Tg_540_

In [2]:
MDDIR='/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD'
for egr in ['OH','SLF']: #
    for nmon,npol in zip([16,80],[30,6]):
        for conc in [0, 0.125]:
            %cd {MDDIR}/PCL_PTM_{egr}/{nmon}mer/{conc}/Tg
            !ls *.gro
            !tail -3 Tg.out
            !tail -20 npt_heating.log 
            


/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0/Tg
npt.gro
imb F  4% pme/F 0.72 step 61619400, will finish Sun Jan 23 06:53:29 2022
imb F  4% pme/F 0.60 step 61619500, will finish Sun Jan 23 06:53:29 2022
imb F  4% pme/F 0.74 step 61619600, will finish Sun Jan 23 06:53:29 2022
 Pres. DC (bar) Pressure (bar)   Constr. rmsd
   -2.84718e+02   -1.35471e+02    1.31202e-06


DD  step 61619699 load imb.: force  6.9%  pme mesh/force 0.520
           Step           Time
       61619700    61619.70000

Current ref_t for group System:    440.0

   Energies (kJ/mol)
           Bond          Angle    Proper Dih. Ryckaert-Bell.          LJ-14
    9.23026e+03    2.95738e+04    9.46400e+02    1.75582e+04    8.48371e+03
     Coulomb-14        LJ (SR)  Disper. corr.   Coulomb (SR)   Coul. recip.
   -4.52793e+04   -2.43010e+04   -1.92148e+03   -1.17670e+04    4.92631e+02
      Potential    Kinetic En.   Total Energy  Conserved En.    Temperature
   -1.69838e+04   

 Pres. DC (bar) Pressure (bar)   Constr. rmsd
   -3.29256e+02   -3.57019e+02    1.34029e-06


DD  step 47440399 load imb.: force  5.4%  pme mesh/force 0.698
           Step           Time
       47440400    47440.40000

Current ref_t for group System:    360.0

   Energies (kJ/mol)
           Bond          Angle    Proper Dih. Ryckaert-Bell.          LJ-14
    9.99644e+03    2.99074e+04    7.99088e+02    2.97273e+04    8.71006e+03
     Coulomb-14        LJ (SR)  Disper. corr.   Coulomb (SR)   Coul. recip.
   -5.20014e+04   -3.04739e+04   -2.49159e+03   -3.79332e+04    6.43607e+02
      Potential    Kinetic En.   Total Energy  Conserved En.    Temperature
   -4.31162e+04    4.54331e+04    2.31692e+03    3.55355e+05    3.63890e+02
 Pres. DC (bar) Pressure (bar)   Constr. rmsd
   -3.27050e+02   -1.49195e+02    1.33627e-06



## Non-equilibrium uniaxial deformation

In [15]:
%%bash
#RUN in beskow terminal for calculating Tg 
#module swap gcc/8.2.0
module load gromacs/2021-avx
MDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD

for egr in OH;do #OH SLF 
    for nmon in 16;do #80
        for conc in 0 ;do #0 0.125
            temp=400;
            #mkdir $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/deform
            cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/deform
            echo PCL_PTM_$egr/${nmon}mer/$conc
            sed -e 's/egrp/'${egr}'/g' -e 's/nmonv/'${nmon}'/g' -e 's/conc/'${conc}'/g' $MDDIR/scripts_mdp/deform.sh > deform.sh
            sed -e 's/temp./'${temp}'/g' $MDDIR/scripts_mdp/neq_deform.mdp > neq_deform.mdp
            mdp=neq_deform.mdp
            top=$MDDIR/PCL_PTM_${egr}/${nmon}mer/${conc}/topol.top
            gmx grompp -f $mdp -c ../400/npt_prod.gro -p $top -o npt_400K.tpr -maxwarn 2
            gmx mdrun -s npt_400K.tpr -deffnm npt_400K -v &
            
            #sed -i 's/nodes=4/nodes=4/' deform.sh
            #sed -i 's/node=32/node=12/' deform.sh
            #sub=$(sbatch deform.sh);id=$(grep -Eo '[0-9]+' <<<"$sub");echo "job submitted at ${tk[t]}: $id"
            #rm \#*
        done
    done
done


PCL_PTM_OH/16mer/0
Setting the LD random seed to -603998233

Generated 210 of the 210 non-bonded parameter combinations

Generated 210 of the 210 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'polymer'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'LI'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'TFS'

turning H bonds into constraints...
Analysing residue names:
There are:   300      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 400 K

Calculated rlist for 1x1 atom pair-list as 1.200 nm, buffer size 0.000 nm

Set rlist, assuming 4x4 atom pair-list, to 1.200 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 12x12x12, spacing 0.399 0.399 0.399

Estima

gcc/8.2.0(22):ERROR:150: Module 'gcc/8.2.0' conflicts with the currently loaded module(s) 'gcc/7.2.0'
gcc/8.2.0(22):ERROR:102: Tcl command execution failed: conflict gcc

openmpi/4.0-gcc-8.2(12):ERROR:151: Module 'openmpi/4.0-gcc-8.2' depends on one of the module(s) 'gcc/8.2.0'
openmpi/4.0-gcc-8.2(12):ERROR:102: Tcl command execution failed: prereq gcc/8.2.0

                       :-) GROMACS - gmx grompp, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Chri

CalledProcessError: Command 'b'#RUN in beskow terminal for calculating Tg \n#module swap gcc/8.2.0\nmodule load gromacs/2021-avx\nMDDIR=/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD\n\nfor egr in OH;do #OH SLF \n    for nmon in 16;do #80\n        for conc in 0 ;do #0 0.125\n            temp=400;\n            #mkdir $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/deform\n            cd $MDDIR/PCL_PTM_$egr/${nmon}mer/$conc/deform\n            echo PCL_PTM_$egr/${nmon}mer/$conc\n            sed -e \'s/egrp/\'${egr}\'/g\' -e \'s/nmonv/\'${nmon}\'/g\' -e \'s/conc/\'${conc}\'/g\' $MDDIR/scripts_mdp/deform.sh > deform.sh\n            sed -e \'s/temp./\'${temp}\'/g\' $MDDIR/scripts_mdp/neq_deform.mdp > neq_deform.mdp\n            mdp=neq_deform.mdp\n            top=$MDDIR/PCL_PTM_${egr}/${nmon}mer/${conc}/topol.top\n            gmx grompp -f $mdp -c ../400/npt_prod.gro -p $top -o npt_bd.tpr -maxwarn 2\n            \n            #sed -i \'s/nodes=4/nodes=4/\' deform.sh\n           

In [4]:
MDDIR='/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD'
for egr in ['OH','SLF']: #
    for nmon in [16,80]:
        for conc in [0, 0.125]:
            %cd {MDDIR}/PCL_PTM_{egr}/{nmon}mer/{conc}/deform
            !tail -3 neq_deform.out
            !tail -15 npt.log 


/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0/deform
starting mdrun 'PCL16_PTM4_OH_0'
10000000 steps,  10000.0 ps.
step 2438400, will finish S   -3.00542e+02    3.17456e+02    1.22860e-06:45 2022

           Step           Time
        2439000     2439.00000

   Energies (kJ/mol)
           Bond          Angle    Proper Dih. Ryckaert-Bell.          LJ-14
    8.02261e+03    2.79585e+04    9.22368e+02    1.69060e+04    8.41596e+03
     Coulomb-14        LJ (SR)  Disper. corr.   Coulomb (SR)   Coul. recip.
   -4.52675e+04   -2.53947e+04   -1.96053e+03   -1.20880e+04    4.18345e+02
      Potential    Kinetic En.   Total Energy  Conserved En.    Temperature
   -2.20668e+04    4.34209e+04    2.13540e+04    1.21354e+05    4.01610e+02
 Pres. DC (bar) Pressure (bar)   Constr. rmsd
   -2.95131e+02    2.46415e+02    1.19813e-06

[Errno 2] No such file or directory: '/cfs/klemming/nobackup/h/hargu978/Cuc_kristina_SelfHealing_polymer/MD/PCL_PTM_OH/16mer/0.